# Exercise 3: Velocity

Aim: To work with velocity data and plot a section

- Author: XX YOUR NAME XX
- Purpose: Plot velocity data
- Date: YYYY-MM-DD

Four figures should be generated:

Packages: You will need to install the package `scipy`

The first time you run through this, 
<hr>

In [ ]:
# Importing required packages here
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import os
import netCDF4
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.dates as mdates
from matplotlib.ticker import ScalarFormatter
from modules.bathymetry import *
from scipy import io



In [ ]:
# Set some paths
# Root directory - change this to the directory where you are working.
rootdir = '/Users/eddifying/Cloudfree/gitlab-cloudfree/messfern-plot-velo/'
#rootdir = 'C:\\Users\\Edward\\Documents\\GitHub\\messfern-plot-velo\\'
os.chdir(rootdir)
datadir = rootdir + 'data/'

# Output file paths for figures
figdir = '../figures/'
if not os.path.exists(figdir):
    os.makedirs(figdir)

# Load the bathymetry data

[ ] Check the code from last week to see how to do this


In [ ]:
# Load the bathymetry data
bathymetry_data_path = datadir + 'bathymetry_subset.nc'

# Add your line of code here

# Print the bathymetry data to verify
#print(bathymetry_data)

# Load the velocity data

In [ ]:
# Load the velocity data from a Matlab file
"""
This script loads velocity data from a Matlab file and extracts specific variables.

Steps:
1. Constructs the file path to the Matlab file.
2. Loads the Matlab file using scipy.io.loadmat.
3. Prints the keys of the loaded data to understand its structure.
4. Extracts latitude, longitude, depth, time, u, and v from the dataset.

Variables:
- inputfile: Path to the Matlab file.
- adcp_data: Dictionary containing the loaded Matlab data.
- ds: Dataset extracted from the Matlab data.
- latitude: Latitude data extracted from the dataset.
- longitude: Longitude data extracted from the dataset.
- depth: Depth data extracted from the dataset.
- time: Time data extracted from the dataset.
- u: U-component of velocity extracted from the dataset.
- v: V-component of velocity extracted from the dataset.

Note:
- The [0][0] indexing is used to access the first element of the nested arrays within the dataset.
"""
inputfile = os.path.join(datadir, 'adcp_DS_msm76.mat')

# Load the Matlab file
adcp_data = io.loadmat(inputfile)

# Print the keys of the loaded data to understand its structure
print(adcp_data.keys())
ds = adcp_data['ds']

# Extract latitude from ds

latitude = ds['lat'][0][0]
longitude = ds['lon'][0][0]
depth = ds['dep'][0][0]
time = ds['time'][0][0]
u = ds['u'][0][0]
v = ds['v'][0][0]

print(f"Size of latitude: {latitude.size}")
print(f"Size of longitude: {longitude.size}")
print(f"Size of depth: {depth.size}")
print(f"Size of time: {time.size}")
print(f"Size of u: {u.size}")
print(f"Size of v: {v.size}")

# Flatten the time, depth, latitude, and longitude arrays
time = time.flatten()

# Convert MATLAB time to datetime64[s]
# This one is incorrect
time_intdays = np.array([np.datetime64('0000-01-01') + np.timedelta64(int(t), 'D') for t in time])
# This one is correct
time = np.array([np.datetime64('0000-01-01') + np.timedelta64(int(t * 86400), 's') for t in time])

depth = depth.flatten()
latitude = latitude.flatten()
longitude = longitude.flatten()

# Turn u into a two dimensional array with length(time) and length(depth)
u = u.reshape(len(depth),len(time))
v = v.reshape(len(depth),len(time))



# Check the original time format

Since we're loading data from a `*.mat` file, it's useful to know that matlab stores time in units of days since 0000-00-00 T 00:00:00 (YYYY-MM-DD T HH:MM:SS).  

Python has a lot of different ways to store time, but in `xarray` it's convenient to use `datetime64`.  Plotting routines know how to handle this data format, and will create nicely formatted time axes.


## Features of this plot

- We've use the function `ax.twinx()` to create a plot with two y-axes.  Here, we're doing it to show the equivalent between the two time variables.


In [ ]:
# Plot the original and the new time as a function of index on two separate axes
fig, ax1 = plt.subplots(figsize=(15, 5))

# Plot original time
ax1.plot(range(len(time)), adcp_data['ds']['time'][0][0].flatten(), label='Original Time', color='blue')
ax1.set_xlabel('Index')
ax1.set_ylabel('Matlab Time', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a second y-axis to plot new time and time_intdays
ax2 = ax1.twinx()
ax2.plot(range(len(time)), time, '-.', label='New Time', color='red')
ax2.plot(range(len(time_intdays)), time_intdays, '--', label='Time in Days', color='green')

ax2.set_ylabel('New Time', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Format the new time axis to show DD-MMM-YYYY
ax2.yaxis.set_major_formatter(mdates.DateFormatter('%d-%b-%Y %H:%M'))

# These next steps are complicated.  See how by running them (change 0 to 1, where 0 means FALSE and 1 means TRUE)
if 0:
    # Set y-axis limits to tight for both axes
    ax2.set_ylim(time_intdays.min(), time.max())
    # Convert the new time limits back to MATLAB time format
    new_time_min = (time_intdays.min() - np.datetime64('0000-01-01')) / np.timedelta64(1, 'D')
    new_time_max = (time.max() - np.datetime64('0000-01-01')) / np.timedelta64(1, 'D')
    # Set y-axis limits for the original time axis
    ax1.set_ylim(new_time_min, new_time_max)

# Set title
fig.suptitle('Original and New Time as a function of Index')

# Add legends
fig.legend(loc='upper right')

# Show the plot
plt.show()

# save the plot
plt.savefig(figdir + 'ex3fig1-Lastname-timeformat.png')

# Optional: Convert the data into xarray and save 

In [ ]:
# Create a new xarray dataset
ds = xr.Dataset(
    {
        'u': (( 'depth', 'time'), u),
        'v': (( 'depth', 'time'), v),
    },
    coords={
        'time': time,
        'depth': depth,
        'latitude': latitude,
        'longitude': longitude,
    },
)

# Define the output path for the NetCDF file
output_path = os.path.join(datadir, 'adcp_DS_msm76.nc')

# Write the dataset to a NetCDF file
ds.to_netcdf(output_path)

print(f"ADCP data written to {output_path}")

## Make a simple plot

### Using Matplotlib, plot velocity

[ ] In the following, can you change the date format on the x-axis to be something more understandable?  What year, month, day, hour are the data from?

In [ ]:
# Initialize the figure and axes
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# Plot u velocity
c1 = ax1.contourf(time, depth, u, cmap='RdBu_r', levels=np.linspace(-1, 1, 21))
fig.colorbar(c1, ax=ax1, orientation='vertical', label='u velocity (m/s)')
ax1.set_ylabel('Depth (m)')
ax1.set_title('u velocity as a function of time and depth')
ax1.invert_yaxis()

# Plot v velocity
c2 = ax2.contourf(time, depth, v, cmap='RdBu_r', levels=np.linspace(-1, 1, 21))
fig.colorbar(c2, ax=ax2, orientation='vertical', label='v velocity (m/s)')
ax2.set_xlabel('Time')
ax2.set_ylabel('Depth (m)')
ax2.set_title('v velocity as a function of time and depth')
ax2.invert_yaxis()

# Set y-axis limits
ax1.set_ylim(600, 0)
ax2.set_ylim(600, 0)

# Set x-axis limits to match the range of the time variable
ax1.set_xlim(time.min(), time.max())
ax2.set_xlim(time.min(), time.max())

# Save the figure
plt.savefig(figdir + 'ex3fig1_LastName-velocity_section.png')
plt.show()


# Now let's figure out where the data are located

The variables latitude and longitude were plotted so let's take a look at those

In [ ]:
# Initialize the figure and axes
fig, ax1 = plt.subplots(figsize=(15, 5))

# Plot latitude as a function of time
ax1.plot(time, latitude, label='Latitude', color='blue')
ax1.set_xlabel('Time')
ax1.set_ylabel('Latitude', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a second y-axis to plot longitude
ax2 = ax1.twinx()
ax2.plot(time, longitude, label='Longitude', color='red')
ax2.set_ylabel('Longitude', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Set title
fig.suptitle('Latitude and Longitude as a function of Time')

# Add legends
fig.legend(loc='upper right')

# Show the plot
plt.show()



### Plot a map using cartopy

- [ ] Update the plot below to add the bathymetry, contoured using the colormap cmap='cmo.deep'.

- [ ] Add a colorbar, experimenting with position using commands like

    cbar = plt.colorbar(contour, ax=ax, orientation='vertical', fraction=0.01, pad=0.04, aspect=30)
    cbar.set_label('Bathymetry (m)')
    cbar.ax.set_position([0.95, 0.5, 0.2, 0.5])

- [ ] Change the extent of the map to match the minimum and maximum values in the bathymetry subset

    min_lon_bathy = bathymetry_subset.lon.min().item()


In [ ]:
# Define the coordinates for the section
section_lat = [latitude[0], latitude[-1]]
section_lon = [longitude[0], longitude[-1]]

# Initialize the figure and axis with Cartopy projection
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add coastlines and gridlines
ax.coastlines()
ax.gridlines(draw_labels=True)

# Plot the section line
ax.plot(section_lon, section_lat, color='red', marker='o', transform=ccrs.PlateCarree(), label='Section')

# Add a legend
plt.legend()

# Set the extent of the map
ax.set_extent([-31.5, -22.5, 63, 68])


# Add features
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)

# Save the figure
plt.savefig(figdir + 'ex3fig2-Lastname-location.png')
plt.show()


## More complicated

using the Gibbs Seawater toolbox (`import gsw`), calculate the distance along the section.

Redo the plots of velocity as a function of distance instead of time.